# Text Classification

* Data Retrieval
* Data Preprocessing and Normalization
* Building Train and Test Datasets
* Feature Engineering Techniques
    1. Traditional
    2. Advanced
* Classification Models
    1. Multinomial Naive Bayes
    2. Logistic Regression
    3. Support Vector Machines
    4. Ensemble Models
    5. Random Forest
    6. Gradient Boosting Machines
* Evaluating Classification Models
    1. Confusion Matrix
* Building and Evaluating Our Text Classifier
    1. Bag of Words Features with Classification Models
    2. TF-IDF Features with Classification Models
    3. Comparative Model Performance Evaluation
    4. Word2Vec Embeddings with Classification Models
    5. GloVe Embeddings with Classification Models
    6. FastText Embeddings with Classification Models
    7. Model Tuning
    8. Model Performance Evaluation

## Data Retrieval

In [24]:
from sklearn.datasets import fetch_20newsgroups
import numpy as np
import nltk
import text_normalizer as tn
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

data = fetch_20newsgroups(subset='all', shuffle=True, remove=('headers', 'footers', 'quotes'))
data_labels_map = dict(enumerate(data.target_names))

In [19]:
# building the dataframe
corpus, target_labels, target_names = (data.data, data.target, [data_labels_map[label] for label in data.target])
data_df = pd.DataFrame({'Article': corpus, 'Target Label': target_labels, 'Target Name': target_names})
print(data_df.shape)
data_df.head(10)

(18846, 3)


,Article,Target Label,Target Name
0,\n\nI am sure some bashers of Pens fans are pr...,10,rec.sport.hockey
1,My brother is in the market for a high-perform...,3,comp.sys.ibm.pc.hardware
2,\n\n\n\n\tFinally you said what you dream abou...,17,talk.politics.mideast
3,\nThink!\n\nIt's the SCSI card doing the DMA t...,3,comp.sys.ibm.pc.hardware
4,1) I have an old Jasmine drive which I cann...,4,comp.sys.mac.hardware
5,\n\nBack in high school I worked as a lab assi...,12,sci.electronics
6,\n\nAE is in Dallas...try 214/241-6060 or 214/...,4,comp.sys.mac.hardware
7,"\n[stuff deleted]\n\nOk, here's the solution t...",10,rec.sport.hockey
8,"\n\n\nYeah, it's the second one. And I believ...",10,rec.sport.hockey
9,\nIf a Christian means someone who believes in...,19,talk.religion.misc


### Data Preprocessing and Normalization

In [20]:
total_nulls = data_df[data_df.Article.str.strip() == ''].shape[0]
print("Empty documents:", total_nulls)

Empty documents: 515


In [21]:
# remove all recrods with no textual content 
data_df = data_df[~(data_df.Article.str.strip() == '')]
data_df.shape

(18331, 3)

In [ ]:
import nltk
stopword_list = nltk.corpus.stopwords.words('english')

# just to keep negation if any in bi-grams
stopword_list.remove('no')
stopword_list.remove('not')

# normalize our corpus
norm_corpus = tn.normalize_corpus(corpus=data_df['Article'], html_stripping=True, contraction_expansion=True, 
                                  accented_char_removal=True, text_lower_case=True, text_lemmatization=True, 
                                  text_stemming=False, special_char_removal=True, remove_digits=True, 
                                  stopword_removal=True, stopwords=stopword_list)

data_df['Clean Article'] = norm_corpus

# view sample data
data_df = data_df[['Article', 'Clean Article', 'Target Label', 'Target Name']]
data_df.head(10)

In [ ]:
data_df = data_df.replace(r'^(\s?)+$', np.nan, regex=True)
data_df.info()